# contextlib

Python的with语句允许我们非常方便地使用资源，而不必担心资源没有关闭，例如：

In [4]:
with open('C:\RSAkey.txt', 'rb') as f:
    f.read()

并不是只有open()函数返回的fp对象才能使用with语句。实际上，任何对象，只要正确实现了上下文管理，就可以用于with语句。

在一个类中，实现上下文管理是通过__enter__和__exit__这两个方法实现的。例如，下面的class实现了这两个方法：

In [5]:
class Query(object):

    def __init__(self, name):
        self.name = name

    def __enter__(self):
        print('Begin')
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type:
            print('Error')
        else:
            print('End')

    def query(self):
        print('Query info about %s...' % self.name)

with Query('Bob') as q:
    q.query()

Begin
Query info about Bob...
End


- @contextmanager

编写`__enter__`和`__exit__`仍然很繁琐，因此Python的标准库contextlib提供了更简单的写法，上面的代码可以改写如下：

In [6]:
from contextlib import contextmanager

class Query(object):

    def __init__(self, name):
        self.name = name

    def query(self):
        print('Query info about %s...' % self.name)

@contextmanager
def create_query(name):
    print('Begin')
    q = Query(name)
    yield q
    print('End')
    
    
with create_query('Bob') as q:
    q.query()

Begin
Query info about Bob...
End


所以可见 `@contextmanager` 也可以实现在某段代码执行的前后自动执行特定的代码。

下面为了加深理解再举个例子：

In [7]:
@contextmanager
def tag(name):
    print("<%s>" % name)
    yield
    print("</%s>" % name)

with tag("h1"):
    print("hello")
    print("world")

<h1>
hello
world
</h1>


代码的执行顺序是：

1. `with`语句首先执行`yield`之前的语句，因此打印出`<h1>`；
2. `yield`调用会执行`with`语句内部的所有语句，因此打印出hello和world；
3. 最后执行`yield`之后的语句，打印出`</h1>`。<br/>
因此，`@contextmanager`让我们通过编写`generator`来简化上下文管理。

- closing

如果一个对象没有实现上下文，我们就不能把它用于with语句。这个时候，可以用closing()来把该对象变为上下文对象。例如，用with语句使用urlopen()：

In [ ]:
from contextlib import closing
from urllib.request import urlopen

with closing(urlopen('https://www.python.org')) as page:
    for line in page:
        print(line)

`closing`也是一个经过`@contextmanager`装饰的`generator`，这个`generator`编写起来其实非常简单：

In [ ]:
@contextmanager
def closing(thing):
    try:
        yield thing
    finally:
        thing.close()

它的作用就是把任意对象变为上下文对象，并支持with语句。

@contextlib还有一些其他decorator，便于我们编写更简洁的代码。